In [ ]:
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
# Construct the base TB model
compartments = (
    "susceptible", 
    "latent_early",
    "latent_late",
    "infectious",
    "treatment", 
    "recovered",
)
tb_model = CompartmentalModel(
    times=[1800.0, 2024.0],
    compartments=compartments,
    infectious_compartments=["infectious"],
)
start_pop = {
    "susceptible": 5e7,
    "infectious": 1.0,
}
tb_model.set_initial_population(start_pop)

# Demographics will be important
tb_model.add_crude_birth_flow(
    "birth",
    Parameter("crude_birth_rate"),
    "susceptible",
)
tb_model.add_universal_death_flows(
    "all_cause_mortality",
    1.0 / Parameter("life_expectancy"),
)
tb_model.request_output_for_compartments("total_population", compartments)

# Infection / reinfection
infection_dest = "latent_early"
tb_model.add_infection_frequency_flow(
    "infection", 
    Parameter("transmission_rate"),
    "susceptible", 
    infection_dest,
)
for source in ["latent_late", "recovered"]:
    reinfection_rate = Parameter("transmission_rate") * Parameter(f"rr_reinfection_{source}")
    tb_model.add_infection_frequency_flow(
        f"reinfection_{source}", 
        reinfection_rate,
        source, 
        infection_dest,
    )

# Latency progression
tb_model.add_transition_flow(
    "stabilisation",
    Parameter("stabilisation_rate"),
    "latent_early",
    "latent_late",
)
for progression_type in ["early", "late"]:
    tb_model.add_transition_flow(
        f"progression_{progression_type}",
        Parameter(f"activation_rate_{progression_type}"),
        f"latent_{progression_type}",
        "infectious",
    )

# Natural history
tb_model.add_transition_flow(
    "self_recovery",
    Parameter("self_recovery_rate"),
    "infectious",
    "recovered",
)
tb_model.add_death_flow(
    "active_tb_death",
    Parameter("tb_death_rate"),
    "infectious",
)

In [ ]:
parameters = {
    "crude_birth_rate": 20.0 / 1000.0,
    "life_expectancy": 60.0,
    "transmission_rate": 0.0,
    "rr_reinfection_latent_late": 1.0,
    "rr_reinfection_recovered": 1.0,
    "stabilisation_rate": 0.0,
    "activation_rate_early": 0.0,
    "activation_rate_late": 0.0,
    "self_recovery_rate": 0.0,
    "tb_death_rate": 0.0,
}
tb_model.run(parameters)

In [ ]:
tb_model.get_derived_outputs_df().plot()